In [80]:
import json
import os

from openai import OpenAI
 
client = OpenAI(
  api_key = "*******" 
)


In [77]:
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)


def get_current_population(location):
    """Get the current population in a given location"""

    population_info = {
        "location": location,
        "population": "100K"
    }
    return json.dumps(population_info)


def get_richness(location):
    """Get the current level of richness in a given location"""
    pil_info = {
        "location": location,
        "richness": "High"
    }
    return json.dumps(pil_info)

def get_most_populated(region = "World"):
    """Get the most populated city"""

    most_populated_byRegion = {"World": "New York", "Europe": "Milan"}

    most_populated = {
        "region": region,
        "location": most_populated_byRegion[region]
    }
        
    return json.dumps(most_populated)

In [52]:
tools = [
        {
            "type": "function",
            "function": {"name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            }}
        },
        {
            "type": "function",
            "function": {"name": "get_current_population",
            "description": "Get the current population of a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    }
                },
                "required": ["location"],
            }}
        },
        {
            "type": "function",
            "function": {"name": "get_richness",
            "description": "Get the current level of richness of a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    }
                },
                "required": ["location"],
            }}
        },
        {
            "type": "function",
            "function": {"name": "get_most_populated",
            "description": "Get the most populated city",
            "parameters": {
                "type": "object",
                "properties": {
                    "region": {
                        "type": "string",
                        "description": "The state or continent where to look for the most populated city, e.g. Europe",
                    }
                },
                "required": [],
            }}
        }
    ]

In [81]:
conversation = [{"role": "system", "content": "You are an AI assistant. You can retrieve specific information using tools that are given to you. You can also use multiple tools to retrieve the complete answer."}]

question = "What is the weather of the most populated city?"

conversation.append({"role": "user", "content": question})
conversation.append({"role": "user", "content": "Given the specified tools, how many sequential steps do you need to answer this question. Write only an integer number for the answer."})

response = client.chat.completions.create(
    model="chat35",
    messages=conversation,
    tools=tools,
    tool_choice="auto"
)

conversation.append({"role": "assistant", "content": response.choices[0].message.content})

steps = int(response.choices[0].message.content)

for i in range(steps):
    conversation.append({"role": "user", "content": question})

    
    response = client.chat.completions.create(
        model="chat35",
        messages=conversation,
        tools=tools,
        tool_choice="auto"
    )

    if response.choices[0].message.tool_calls:
        for tool_call in response.choices[0].message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            if function_name == 'get_current_weather':
                function_response = get_current_weather(**function_args)
                conversation.append({"role": "function", "name": function_name,"content": function_response})
            elif function_name == 'get_current_population':
                function_response = get_current_population(**function_args)
                conversation.append({"role": "function", "name": function_name,"content": function_response})
            elif function_name == 'get_richness':
                function_response = get_richness(**function_args)
                conversation.append({"role": "function", "name": function_name,"content": function_response})
            elif function_name == 'get_most_populated':
                function_response = get_most_populated(**function_args)
                conversation.append({"role": "function", "name": function_name,"content": function_response})

    else:
        conversation.append({"role": "assistant", "content": response.choices[0].message.content})

conversation.append({"role": "user", "content": "Summarize with a final answer."})


response = client.chat.completions.create(
    model="chat35",
    messages=conversation,
    tools=tools,
    tool_choice="auto"
)

conversation.append({"role": "assistant", "content": response.choices[0].message.content})


In [82]:
conversation

[{'role': 'system',
  'content': 'You are an AI assistant. You can retrieve specific information using tools that are given to you. You can also use multiple tools to retrieve the complete answer.'},
 {'role': 'user',
  'content': 'What is the weather of the most populated city?'},
 {'role': 'user',
  'content': 'Given the specified tools, how many sequential steps do you need to answer this question. Write only an integer number for the answer.'},
 {'role': 'assistant', 'content': '2'},
 {'role': 'user',
  'content': 'What is the weather of the most populated city?'},
 {'role': 'function',
  'name': 'get_most_populated',
  'content': '{"region": "World", "location": "New York"}'},
 {'role': 'user',
  'content': 'What is the weather of the most populated city?'},
 {'role': 'function',
  'name': 'get_current_weather',
  'content': '{"location": "New York, NY", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'},
 {'role': 'user', 'content': 'Summarize with a fin